In [ ]:
#upload the transformer and label encoding files first

In [1]:
!pip install elephas
!pip install streamlit
!pip install pyngrok==4.1.1
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.1 MB 7.6 MB/s 
     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 51.9 MB/s 
  Created wheel for elephas: filename=elephas-3.1.0-py3-none-any.whl size=26259 sha256=dcd61a02338aac31d720e543dd6e1a2ed85a0c970d84163da4936d68d97e8910
  Stored in directory: /root/.cache/pip/wheels/33/e4/e1/56dda8be927bb0e9971cd7ddf3fc1b17ce78db56268b1f867f
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=53f423f4c82163ebfa851a63acc4904c70e8af0841817044ddceb37e3f128962
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built elephas pyspark
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
Looking in indexes: htt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=e384bc510499db175e729e5b2d5df0f01ea89fbc79ada5ee26070e46fffcd381
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of 

In [2]:
!ngrok authtoken {'296gloYMZx0IEn84mbVMCKzo1SV_6Sp5sYcPoFygBrSxhqrgs'}

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


##Streamlit App

In [3]:
%%writefile app.py
#importing the libraries
import streamlit as st
import joblib
from PIL import Image
from skimage.transform import resize
import numpy as np
import time
from elephas.ml_model import ElephasTransformer,load_ml_transformer
import pyspark
from pyspark.sql import SparkSession
from tensorflow.keras.preprocessing.image import img_to_array
from elephas.ml.adapter import to_data_frame
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql.functions import col
from elephas.utils.model_utils import argmax
import pickle

#start spark session

sc = pyspark.SparkContext.getOrCreate();
spark = SparkSession.builder.getOrCreate()


# Designing the interface
st.title("Plant disease Identification App")
# For newline
st.write('\n')

st.sidebar.title("Upload Image")

#Disabling warning
st.set_option('deprecation.showfileUploaderEncoding', False)

#Choose your own image and display Images
uploaded_file = st.sidebar.file_uploader(" ",type=['png', 'jpg', 'jpeg'],accept_multiple_files=True)
if uploaded_file is not None:
  if len(uploaded_file)<2:
    st.warning('Minimum images to upload is 2. Upload the same image twice if you have only one image.')
  for images_id, image_name in enumerate(uploaded_file):
    image = Image.open(uploaded_file[images_id])
    st.sidebar.image(image,caption=str(image_name))

#process input images for prediction
def get_prediction_label(list_pred,label_data):
  round_pred_list=[round(num, 3) for num in list_pred]
  max_value = max(round_pred_list)
  max_index = round_pred_list.index(max_value)
  pred="{} with {}% confidence.".format(label_data[max_index],max_value*100)
  return pred

if st.button('Predict'):
  if (uploaded_file is not None) and len(uploaded_file)>1: 
    st.write('Please wait, the predictions are being generated')
    final_image_list=[]
    final_label=[]
    name_list=[]
    for images_id, image_name in enumerate(uploaded_file):
      image = Image.open(uploaded_file[images_id]).resize([24,24])
      image =img_to_array(image,dtype='float16')/255
      image=image.flatten().tolist()
      final_image_list.append(image)
      final_label.append(0)
      name_list.append(str(image_name.name))
      # pyspark datframe
    features = np.asarray(final_image_list)
    labels = np.asarray(final_label)
    data_frame = to_data_frame(sc, features, final_label)
    list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
    data_frame = data_frame.select(list_to_vector_udf(data_frame["features"]).alias("features"))
    #load predictor
    loaded_transformer = load_ml_transformer("/content/colab_elephas_transformer_2june_v4.h5")
    #predict
    d=loaded_transformer.transform(data_frame)
    d = d.drop("features")
    d=d.toPandas()
    d['Image name']=name_list
    a_file = open("/content/inal_label_to_disease_2_june_v4", "rb")
    label_data= pickle.load(a_file)
    label_data = {v: k for k, v in label_data.items()}
    d['prediction']=d['prediction'].apply(lambda x: get_prediction_label(x,label_data))
    d = d[['Image name','prediction']]
    st.table(d)
    @st.cache
    def convert_df(df):
      return df.to_csv().encode('utf-8')

    csv = convert_df(d)

    st.download_button(
      "Download Predictions",
      csv,
      "file.csv",
      "text/csv",
      key='download-csv'
    )
  else:
    st.warning('Please upload the images first')





Writing app.py


In [4]:
#run streamlit app
!streamlit run /content/app.py &>/dev/null&


In [5]:
#kill any ngrok seesions alreday running
!killall ngrok

ngrok: no process found


In [6]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
print(public_url)


http://5091-34-75-95-216.ngrok.io
